In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

import dateutil.parser

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [113]:
# Get dictionary from scrape

import pickle
with open("../data/interim/0201_private_prop_dict.pkl", 'rb') as picklefile: 
    priv_dict = pickle.load(picklefile)
print(priv_dict)

{2411722240007: {'address': '222 9th St S', 'property_id': 2411722240007, 'construction_year': '1922', 'est_market_value': '$169,000', 'owner_name': 'MARY JANE GRAPER', 'building_market_value': '$80,000', 'tax_market_value': '$146,970', 'homestead_status': 'HOMESTEAD', 'total_market_value': '$169,000', 'total_tax': '', 'homestead_mrkt_val_excl': '$22,030', 'total_net_tax': '$2,524.34', 'land_market_value': '$89,000', 'property_type': 'RESIDENTIAL'}, 1602924210184: {'address': '2410 Irving Ave N', 'property_id': 1602924210184, 'construction_year': '1968', 'est_market_value': '', 'owner_name': 'BOYS & GIRLS CLUB OF TC', 'building_market_value': '', 'tax_market_value': '', 'homestead_status': 'NON-HOMESTEAD', 'total_market_value': '', 'total_tax': '', 'homestead_mrkt_val_excl': '', 'total_net_tax': '', 'land_market_value': '', 'property_type': 'COMMERCIAL PREFERRED'}, 2202924140554: {'address': '400 1st Ave N', 'property_id': 2202924140554, 'construction_year': '1900', 'est_market_value':

In [114]:
# Create a dataframe from the dictionary
priv_df = pd.DataFrame.from_dict(priv_dict, orient="index")

In [115]:
priv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 102824110123 to 3602924430088
Data columns (total 17 columns):
address                    838 non-null object
property_id                838 non-null int64
construction_year          838 non-null object
est_market_value           838 non-null object
owner_name                 838 non-null object
building_market_value      838 non-null object
tax_market_value           838 non-null object
homestead_status           838 non-null object
total_market_value         838 non-null object
total_tax                  838 non-null object
homestead_mrkt_val_excl    838 non-null object
total_net_tax              838 non-null object
land_market_value          838 non-null object
property_type              838 non-null object
sale_date                  309 non-null object
sale_price                 309 non-null object
transaction_type           309 non-null object
dtypes: int64(1), object(16)
memory usage: 117.8+ KB


In [116]:
priv_df.head()

,address,property_id,construction_year,est_market_value,owner_name,building_market_value,tax_market_value,homestead_status,total_market_value,total_tax,homestead_mrkt_val_excl,total_net_tax,land_market_value,property_type,sale_date,sale_price,transaction_type
102824110123,2730 31St St E,102824110123,1923,,HOLY TRINITY LUTH CH MPLS,,,NON-HOMESTEAD,,,,,,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824210283,3106 Nicollet Ave,302824210283,1989,,METRO TRANSIT COMMISSION,,,NON-HOMESTEAD,,,,,,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824240197,3335 Blaisdell Ave,302824240197,1961,,YOUNG MENS CHRISTIAN ASN T/C,,,NON-HOMESTEAD,,,,,,COMMERCIAL PREFERRED,NaN,NaN,NaN
302924210025,3660 Technology Dr,302924210025,1985,,CITY OF MINNEAPOLIS,,,NON-HOMESTEAD,,,,,,COMMERCIAL PREFERRED,"May, 2009","$6,100,000",Warranty Deed
402824210016,1455 Lake St W,402824210016,1961,"$4,050,000",SONS OF NORWAY,"$3,195,600","$4,050,000",NON-HOMESTEAD,"$4,050,000",,,"$161,031.14","$854,400",COMMERCIAL PREFERRED,NaN,NaN,NaN


In [117]:
# Remove empty strings

def replace_empty_values(df):
    df_cl = df.replace(r'\s+', np.nan, regex=True)
    df_cl = df.replace('', np.nan)
    return df_cl

priv_df_cl = replace_empty_values(priv_df)

In [118]:
# First converted empty strings to NaN

# priv_df_cl['building_market_value'].replace('[\$,)]','', regex=True).astype(float)
priv_df_cl

,address,property_id,construction_year,est_market_value,owner_name,building_market_value,tax_market_value,homestead_status,total_market_value,total_tax,homestead_mrkt_val_excl,total_net_tax,land_market_value,property_type,sale_date,sale_price,transaction_type
102824110123,2730 31St St E,102824110123,1923,NaN,HOLY TRINITY LUTH CH MPLS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824210283,3106 Nicollet Ave,302824210283,1989,NaN,METRO TRANSIT COMMISSION,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824240197,3335 Blaisdell Ave,302824240197,1961,NaN,YOUNG MENS CHRISTIAN ASN T/C,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302924210025,3660 Technology Dr,302924210025,1985,NaN,CITY OF MINNEAPOLIS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,"May, 2009","$6,100,000",Warranty Deed
402824210016,1455 Lake St W,402824210016,1961,"$4,050,000",SONS OF NORWAY,"$3,195,600","$4,050,000",NON-HOMESTEAD,"$4,050,000",NaN,NaN,"$161,031.14","$854,400",COMMERCIAL PREFERRED,NaN,NaN,NaN
502824120005,3033 Excelsior Blvd,502824120005,1955,"$22,820,000",ACKY-CALHOUN LLC,"$16,349,900","$22,820,000",NON-HOMESTEAD,"$22,720,000",NaN,NaN,"$913,934.73","$6,370,100",COMMERCIAL PREFERRED,"August, 2000","$24,500,000",Warranty Deed
502924130012,4021 Thomas Ave N,502924130012,1950,"$180,000",MCA HOLDINGS LLC,NaN,"$180,000",NON-HOMESTEAD,"$180,000",NaN,NaN,"$5,765.46","$180,000",LAND COMMERCIAL PREFERRED,NaN,NaN,NaN
511722140018,810 Glenwood Ave N,511722140018,1992,"$572,400",A L VERPY & P A VERPY,"$392,200","$572,400",HOMESTEAD,"$572,400",NaN,NaN,"$8,285.40","$180,200",RESIDENTIAL,"June, 1995","$371,000",NaN
511722320050,1101 Lasalle Ave,511722320050,1966,"$367,000",AM PARTNERS LLC,"$166,000","$367,000",NON-HOMESTEAD,"$367,000",NaN,NaN,"$5,756.66","$201,000",RESIDENTIAL TWO UNIT,"April, 2005","$450,000",Warranty Deed
1002824220092,3800 Pleasant Ave,1002824220092,1934,NaN,SAGRADO CORAZON DE JESUS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN


In [123]:
# Clean and convert currency fields

def clean_scrape_data(df):
    df['building_market_value'].replace('[\$,)]','', inplace=True, regex=True)
    df['building_market_value'] = df['building_market_value'].astype(float, copy=False)
    
    df['homestead_mrkt_val_excl'].replace('[\$,)]','', inplace=True, regex=True)
    df['homestead_mrkt_val_excl'] = df['homestead_mrkt_val_excl'].astype(float, copy=False)
    
    df['total_market_value'].replace('[\$,)]','', inplace=True, regex=True)
    df['total_market_value'] = df['total_market_value'].astype(float, copy=False)
    
    df['total_tax'].replace('[\$,)]','', inplace=True, regex=True)
    df['total_tax'] = df['total_tax'].astype(float, copy=False)
    
    df['est_market_value'].replace('[\$,)]','', inplace=True, regex=True)
    df['est_market_value'] = df['est_market_value'].astype(float, copy=False)
    
    df['land_market_value'].replace('[\$,)]','', inplace=True, regex=True)
    df['land_market_value'] = df['land_market_value'].astype(float, copy=False)
    
    df['sale_price'].replace('[\$,)]','', inplace=True, regex=True)
    df['sale_price'] = df['sale_price'].astype(float, copy=False)
    
    df['tax_market_value'].replace('[\$,)]','', inplace=True, regex=True)
    df['tax_market_value'] = df['tax_market_value'].astype(float, copy=False)
    
    df['total_net_tax'].replace('[\$,)]','', inplace=True, regex=True)
    df['total_net_tax'] = df['total_net_tax'].astype(float, copy=False)

In [124]:
clean_scrape_data(priv_df_cl)

In [125]:
priv_df_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 102824110123 to 3602924430088
Data columns (total 17 columns):
address                    838 non-null object
property_id                838 non-null int64
construction_year          801 non-null object
est_market_value           762 non-null float64
owner_name                 838 non-null object
building_market_value      741 non-null float64
tax_market_value           762 non-null float64
homestead_status           838 non-null object
total_market_value         762 non-null float64
total_tax                  0 non-null float64
homestead_mrkt_val_excl    167 non-null float64
total_net_tax              762 non-null float64
land_market_value          762 non-null float64
property_type              838 non-null object
sale_date                  309 non-null object
sale_price                 309 non-null float64
transaction_type           281 non-null object
dtypes: float64(9), int64(1), object(7)
memory usage: 117.8+ KB


In [122]:
priv_df_cl


,address,property_id,construction_year,est_market_value,owner_name,building_market_value,tax_market_value,homestead_status,total_market_value,total_tax,homestead_mrkt_val_excl,total_net_tax,land_market_value,property_type,sale_date,sale_price,transaction_type
102824110123,2730 31St St E,102824110123,1923,NaN,HOLY TRINITY LUTH CH MPLS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824210283,3106 Nicollet Ave,302824210283,1989,NaN,METRO TRANSIT COMMISSION,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824240197,3335 Blaisdell Ave,302824240197,1961,NaN,YOUNG MENS CHRISTIAN ASN T/C,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302924210025,3660 Technology Dr,302924210025,1985,NaN,CITY OF MINNEAPOLIS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,"May, 2009",6100000.0,Warranty Deed
402824210016,1455 Lake St W,402824210016,1961,4050000.0,SONS OF NORWAY,3195600.0,"$4,050,000",NON-HOMESTEAD,4050000.0,NaN,NaN,"$161,031.14",854400.0,COMMERCIAL PREFERRED,NaN,NaN,NaN
502824120005,3033 Excelsior Blvd,502824120005,1955,22820000.0,ACKY-CALHOUN LLC,16349900.0,"$22,820,000",NON-HOMESTEAD,22720000.0,NaN,NaN,"$913,934.73",6370100.0,COMMERCIAL PREFERRED,"August, 2000",24500000.0,Warranty Deed
502924130012,4021 Thomas Ave N,502924130012,1950,180000.0,MCA HOLDINGS LLC,NaN,"$180,000",NON-HOMESTEAD,180000.0,NaN,NaN,"$5,765.46",180000.0,LAND COMMERCIAL PREFERRED,NaN,NaN,NaN
511722140018,810 Glenwood Ave N,511722140018,1992,572400.0,A L VERPY & P A VERPY,392200.0,"$572,400",HOMESTEAD,572400.0,NaN,NaN,"$8,285.40",180200.0,RESIDENTIAL,"June, 1995",371000.0,NaN
511722320050,1101 Lasalle Ave,511722320050,1966,367000.0,AM PARTNERS LLC,166000.0,"$367,000",NON-HOMESTEAD,367000.0,NaN,NaN,"$5,756.66",201000.0,RESIDENTIAL TWO UNIT,"April, 2005",450000.0,Warranty Deed
1002824220092,3800 Pleasant Ave,1002824220092,1934,NaN,SAGRADO CORAZON DE JESUS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN


In [126]:
with open('0201_prop_info_data.pkl', 'wb') as picklefile:
    pickle.dump(priv_df_cl, picklefile)

In [128]:
# Create another smaller dataframe with unneeded columns removed

priv_df_cl_abr = priv_df_cl[['address', 'property_id']]
priv_df_cl_abr.head()

,address,property_id
102824110123,2730 31St St E,102824110123
302824210283,3106 Nicollet Ave,302824210283
302824240197,3335 Blaisdell Ave,302824240197
302924210025,3660 Technology Dr,302924210025
402824210016,1455 Lake St W,402824210016


In [129]:
with open('../data/processed/0201_prop_info_data_abr.pkl', 'wb') as picklefile:
    pickle.dump(priv_df_cl_abr, picklefile)